In [72]:
# coding:utf-8
'''
抓取中国汽车燃料消耗量查询系统数据
网址：http://www.miit.gov.cn/asopCmsSearch/n2068/index.html?searchId=scqycx
'''

import requests
import time



# 设置request headers
headers = {
    'Host': 'www.miit.gov.cn',
    'Connection': 'keep-alive',
    'Accept': 'text/javascript, application/javascript, */*',
    'x-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.6 Safari/537.36',
    'Referer': 'http://www.miit.gov.cn/asopCmsSearch/n2257/n2280/index.html',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cookie': 'JSESSIONID=96D5B04AD84EA1FEC31C2AF829273263; Hm_lvt_af6f1f256bb28e610b1fc64e6b1a7613=1569584885'
}



url_head = "http://www.miit.gov.cn/asopCmsSearch/searchIndex.jsp?params=%257B%2522goPage%2522%253A"
url_page = 10
url_foot = "%252C%2522orderBy%2522%253A%255B%257B%2522orderBy%2522%253A%2522pl%2522%252C%2522reverse%2522%253Afalse%257D%255D%252C%2522pageSize%2522%253A10%252C%2522queryParam%2522%253A%255B%257B%2522shortName%2522%253A%2522allRecord%2522%252C%2522value%2522%253A%25221%2522%257D%255D%257D&callback=jsonp"


"""使用requests请求代理服务器，适用于http，https"""

#要访问的目标网页
# page_url = "http://dev.kdlapi.com/testproxy/"
url_time = int(round(time.time() * 1000))
url = url_head + str(url_page) + url_foot + str(url_time)

#代理服务器
proxy = "122.114.125.90:16819"

#用户名和密码(私密代理/独享代理)
username = "841414840"
password = "jhlahb8c"

proxies = {
    "http": "http://%(user)s:%(pwd)s@%(proxy)s/" % {'user': username, 'pwd': password, 'proxy': proxy},
    "https": "http://%(user)s:%(pwd)s@%(proxy)s/" % {'user': username, 'pwd': password, 'proxy': proxy}
}


response = requests.get(url=url, proxies=proxies, headers=headers)
response.close()
print(response)
if response.status_code == 200:
    response = response.text.strip()
    print(response)
 





<Response [200]>
jsonp1570677879744({"class":"class com.assoft.search.model.QueryResult","curPage":200,"resultMap":[{"sjgk":"6.5","clxh":"LQG5010CSLNF","scqy":"柳州五菱专用汽车制造有限公司","sqgk":"8.9","clzl":"轻型货车N1类","zhgk":"7.4","baID":"20100901019221","sygjbz":"GB 20997—2008","tgrq":"20100901000000","pl":"998","zdsjzzl":"1430","sjlrsj":"20100901000000","zczbzl":"810","edgl":"45","fdjxh":"LJ465QR1E2","qdxs":"后轮驱动","bsqlx":"MT","rllx":"汽油"},{"sjgk":"5.7","clxh":"ZQ6385A62F","scqy":"海马商务汽车有限公司","sqgk":"8.4","clzl":"乘用车M1类","zhgk":"6.7","baID":"20101008020375","sygjbz":"GB 19578—2004","tgrq":"20101001000000","tymc":"福仕达","pl":"998","sjlrsj":"20101001000000","zdsjzzl":"1660","zczbzl":"1095","edgl":"45","qdxs":"后轮驱动","fdjxh":"LJ465QR1E2","bsqlx":"MT","rllx":"汽油"},{"sjgk":"6.2","clpp":"金杯","clxh":"SY5020XXH-A6SBW","scqy":"沈阳华晨金杯汽车有限公司","clcd":"国产","sqgk":"9.5","clzl":"轻型货车N1类","zhgk":"7.2","baID":"20101008020012","sygjbz":"GB 20997—2008","tgrq":"20101001000000","tymc":"金杯 海星","pl":"998","sjlrsj":"2010

In [68]:
import pymysql
# 每条油耗数据存入数据库
def insert_into_MySQL(lists):
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='MySQL', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE miit_gov_fuel_consumption")
    data_tuple = tuple(lists)
    try:
        sql = """
            INSERT INTO `fuel_consumption_info` 
            (`市郊工况`,`车辆型号`,`生产企业`,`市区工况`,`综合工况`,`车辆种类`,`备案号`,`使用国家标准`,`通告日期`,`通用名称`,`最大设计总质量`,`实际录入时间`,`整车整备质量`,`驱动形式`,`变速器类型`,`发动机型号`,`额定功率`,`燃料类型`) 
            VALUES{};
        """.format(data_tuple)
        
        cur.execute(sql)
        
    finally:
        conn.commit()
        cur.close()
        conn.close()

# 每页原始网页数据存储
def page_source_insert_into_MySQL(tuples):
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='MySQL', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE miit_gov_fuel_consumption")
    try:
        sql = """
            INSERT INTO `fuel_consumption_response` 
            (`url`,`page`,`response`) 
            VALUES
            {};""".format(tuples)
        print(sql)
        cur.execute(sql)
        conn.commit()
    finally:
        cur.close()
        conn.close()
        
# 原始网站数据存储
page_source_tuple = tuple([url,str(url_page),response])
print(page_source_tuple)
print(type(page_source_tuple))
page_source_insert_into_MySQL(page_source_tuple)


# 网页返回结果处理为字典形式
response_dict = eval(response[19:-2])
# 提取字典中的油耗信息
page_info = response_dict['resultMap']


# 插入数据库字段属性顺序列表
# ['市郊工况','车辆型号','生产企业','市区工况','综合工况','车辆种类','备案号','使用国家标准','通告日期','通用名称','最大设计总质量','实际录入时间','整车整备质量','驱动形式','变速器类型','发动机型号','额定功率','燃料类型']
get_info_list =['sjgk','clxh','scqy','sqgk','zhgk','clzl','baID','sygjbz','tgrq','tymc','zdsjzzl','sjlrsj','zczbzl','qdxs','bsqlx','fdjxh','edgl','rllx']

# 初始化每页结果列表
page_result_list = []
# 遍历每页油耗信息，每条记录存入单个列表(字典形式提取键值)，并加入每页结果列表
for single in page_info:
    # 初始化每条记录存储列表
    single_info_list = []
    # 遍历需要获取的属性值列表
    for attr in get_info_list:
        attr_value = single[attr]
        single_info_list.append(attr_value)
    page_result_list.append(single_info_list)


# 采集的每页所有记录插入数据库
for i in page_result_list:
    print(i)
    insert_into_MySQL(i)


('http://www.miit.gov.cn/asopCmsSearch/searchIndex.jsp?params=%257B%2522goPage%2522%253A2%252C%2522orderBy%2522%253A%255B%257B%2522orderBy%2522%253A%2522pl%2522%252C%2522reverse%2522%253Afalse%257D%255D%252C%2522pageSize%2522%253A10%252C%2522queryParam%2522%253A%255B%257B%2522shortName%2522%253A%2522allRecord%2522%252C%2522value%2522%253A%25221%2522%257D%255D%257D&callback=jsonp1570675259329', '2', 'jsonp1570675259329({"class":"class com.assoft.search.model.QueryResult","curPage":2,"resultMap":[{"sjgk":"7.6","clxh":"SY6521D4S1BG3","scqy":"华晨雷诺金杯汽车有限公司","sqgk":"12.6","zhgk":"9.6","clzl":"乘用车M1类","baID":"20180917111494","sygjbz":"GB 19578—2014","tgrq":"20180901000000","tymc":"阁瑞斯","zdsjzzl":"2800","sjlrsj":"20180901000000","zczbzl":"1950","qdxs":"后轮驱动","bsqlx":"MT","fdjxh":"V19","edgl":"78","rllx":"汽油"},{"sjgk":"7.6","clxh":"SY6521D4S1BG5","scqy":"华晨雷诺金杯汽车有限公司","sqgk":"12.6","clzl":"乘用车M1类","zhgk":"9.6","baID":"20180917111498","sygjbz":"GB 19578—2014","tgrq":"20180901000000","tymc":"阁瑞斯"

['7.6', 'SY6513S2Z1BG', '华晨雷诺金杯汽车有限公司', '12.7', '9.6', '乘用车M1类', '20181217113793', 'GB 19578—2014', '20181201000000', '华颂', '2715', '20181201000000', '2110', '后轮驱动', 'AT', 'N20B20C', '178', '汽油']
['7.9', 'SY6513S1Z1BG', '华晨雷诺金杯汽车有限公司', '13.4', '9.6', '乘用车M1类', '20181217113794', 'GB 19578—2014', '20181201000000', '华颂', '2575', '20181201000000', '2050', '后轮驱动', 'AT', 'N20B20C', '150', '汽油']
['7.9', 'SY5033XQC-S1Z1BG', '华晨雷诺金杯汽车有限公司', '13.4', '9.6', '乘用车M1类', '20181217113795', 'GB 19578—2014', '20181201000000', '华颂', '2575', '20181201000000', '2015', '后轮驱动', 'AT', 'N20B20C', '150', '汽油']
['8', 'SY6534D4S1BH2', '华晨雷诺金杯汽车有限公司', '9.8', '8.8', '乘用车M1类', '20181217113803', 'GB 19578—2014', '20181201000000', '金杯牌', '2930', '20181201000000', '1750', '后轮驱动', 'MT', 'V19', '78', '汽油']


In [69]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9
